In [1]:
# to import MPIManager
using MPIClusterManagers

# need to also import Distributed to use addprocs()
using Distributed

# specify, number of mpi workers, launch cmd, etc.
manager=MPIManager(np=4)

# start mpi workers and add them as julia workers too.
addprocs(manager)

4-element Vector{Int64}:
 2
 3
 4
 5

In [2]:
@mpi_do manager begin
    using MPI: MPI, Comm, Win, free
    comm = MPI.COMM_WORLD
    rank = MPI.Comm_rank(comm)
    size = MPI.Comm_size(comm)
    println("Hello world, I am $(rank) of $(size)")
end

      From worker 5:	Hello world, I am 3 of 4
      From worker 4:	Hello world, I am 2 of 4
      From worker 3:	Hello world, I am 1 of 4
      From worker 2:	Hello world, I am 0 of 4


In [6]:
using DistributedArrays
@mpi_do manager begin
    using DistributedArrays
end

In [33]:
a = 1

1

In [34]:
@mpi_do manager begin
    println(a)
end

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 5:
    UndefVarError: a not defined
    Stacktrace:
     [1] top-level scope
    [90m   @ [39m[90;4mIn[34]:2[0m
     [2] [0m[1meval[22m
    [90m   @ [39m[90m./[39m[90;4mboot.jl:360[0m[90m [inlined][39m
     [3] [0m[1m#77[22m
    [90m   @ [39m[90m~/.julia/packages/MPIClusterManagers/0ZYYQ/src/[39m[90;4mmpimanager.jl:510[0m
     [4] [0m[1m#103[22m
    [90m   @ [39m[90m/Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mprocess_messages.jl:274[0m
     [5] [0m[1mrun_work_thunk[22m
    [90m   @ [39m[90m/Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mprocess_messages.jl:63[0m
     [6] [0m[1mrun_work_thunk[22m
    [90m   @ [39m[90m/Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mprocess_messages.jl:72[0m
 

# MPI + Distributed Arrays

([Back to Overview](../index.html#/0/7))

In [53]:
@mpi_do manager begin
function work(N)
    x0 = rank
    for n = 1:Int(N)
        x0 = sqrt(x0 + 1)
    end
    out = zeros(2, 2)
    out[1, 1] = x0 + rank
    return out
end
end

In [57]:
r1 = DistributedArrays.remotecall(() -> work(1e10), workers()[1]) 
r2 = DistributedArrays.remotecall(() -> work(1e10), workers()[2]) 
r3 = DistributedArrays.remotecall(() -> work(1e10), workers()[3]) 
r4 = DistributedArrays.remotecall(() -> work(1e10), workers()[4]) 
D  = DArray(reshape([r1 r2 r3 r4], (2,2))) 

4×4 DArray{Float64, 2, Matrix{Float64}}:
 1.61803  0.0  3.61803  0.0
 0.0      0.0  0.0      0.0
 2.61803  0.0  4.61803  0.0
 0.0      0.0  0.0      0.0

In [14]:
[@fetchfrom p localindices(D) for p in workers()]

4-element Vector{Tuple{UnitRange{Int64}, UnitRange{Int64}}}:
 (1:2, 1:2)
 (3:4, 1:2)
 (1:2, 3:4)
 (3:4, 3:4)

In [27]:
@mpi_do manager begin
    if rank == 2
        println(D)
    end
end

      From worker 4:	[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [19]:
localindices(D)

(1:0, 1:0)